# 加载Fashion-MNIST作为训练数据集

In [73]:
from mxnet import gluon
from mxnet import nd
from mxnet import autograd
import mxnet as mx
from mxnet.gluon import nn 
import d2l 
def transform(data, label): return data.astype(
    'float32')/255, label.astype('float32')
dataset = gluon.data.vision
trans = []
trans.append(dataset.transforms.ToTensor())
trans = dataset.transforms.Compose(trans)

# minist_train = gluon.data.vision.FashionMNIST(
#     root='fashion-mnist/', train=True, transform=transform)
# minist_test = gluon.data.vision.FashionMNIST(
#     root='fashion-mnist/', train=False, transform=transform)
mnist_train = dataset.FashionMNIST(root='fashion-mnist/',train=True).transform_first(trans)
mnist_test = dataset.FashionMNIST(root='fashion-mnist/',train=False).transform_first(trans)
batch_size = 256
train_iter = gluon.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_iter = gluon.data.DataLoader(mnist_test, batch_size, shuffle=True)
for data, label in train_iter:
    print(data.shape)
    break


(256, 1, 28, 28)


In [74]:
# ?? gluon.Trainer

# 建议使用GPU来加速计算

In [75]:
def try_gpu():
    try:
        ctx=mx.gpu()
        _=nd.zeros((1),ctx=ctx)
    except mx.base.MXNetError:
        ctx=mx.cpu()
    return ctx
ctx=try_gpu()
ctx

cpu(0)

# 通过`Sequential`类来实现LeNet模型

In [76]:
from mxnet.gluon import loss as gloss,nn
from mxnet import init 
net=nn.Sequential() 
net.add(nn.Conv2D(channels=6,kernel_size=5,activation='sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       nn.Conv2D(channels=16,kernel_size=5,activation='sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       nn.Dense(120,activation='sigmoid'),
       nn.Dense(84,activation='sigmoid'),
       nn.Dense(10))
net.initialize(force_reinit=True,ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})

In [77]:
from mxnet import nd 
X=nd.random.uniform(shape=(1,1,28,28))
for layer in net:
    X=layer(X)
    print(layer.name,' output shape:  ',X.shape)

conv8  output shape:   (1, 6, 24, 24)
pool8  output shape:   (1, 6, 12, 12)
conv9  output shape:   (1, 16, 8, 8)
pool9  output shape:   (1, 16, 4, 4)
dense12  output shape:   (1, 120)
dense13  output shape:   (1, 84)
dense14  output shape:   (1, 10)


# `as_in_context` 函数将数据复制到显存上

In [78]:
def evaluate_accuracy(data_iter,net,ctx):
    acc_sum,n=nd.array([0],ctx=ctx),0
    for X,y in data_iter:
        X,y=X.as_in_context(ctx),y.as_in_context(ctx).astype('float32')
        acc_sum+=((net(X).argmax(axis=1)==y)).sum()
        n+=y.size
    return acc_sum.asscalar()/n 

In [79]:
import time 
def train(train_iter, test_iter, epoches, trainer, net, batch_size, ctx):
    loss=gloss.SoftmaxCrossEntropyLoss()
    for e in range(epoches):
        train_acc, train_loss, test_acc ,n,start= .0, .0, .0,.0,time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                yhat = net(X)
                l = loss(yhat, y).sum()
            l.backward()
            y = y.astype('float32')
            trainer.step(batch_size)
            train_acc+=(yhat.argmax(axis=1)==y).sum().asscalar() 
            train_loss=l.asscalar()
            n+=y.size
        test_acc=evaluate_accuracy(test_iter,net,ctx)
        print('epoechs:%d, loss: %.4f, train_acc:%.3f, test_acc:%.3f ,used time: %.1f sec'%(e+1,train_loss/n,train_acc/n,test_acc,time.time()-start))
            

In [80]:
train(train_iter, test_iter, 5, trainer, net, batch_size, ctx)

epoechs:1, loss: 0.0037, train_acc:0.101, test_acc:0.100 ,used time: 20.9 sec
epoechs:2, loss: 0.0037, train_acc:0.100, test_acc:0.100 ,used time: 20.1 sec
epoechs:3, loss: 0.0037, train_acc:0.101, test_acc:0.100 ,used time: 19.8 sec


KeyboardInterrupt: 

In [ ]:
d=net[2]
d.weight